In [1]:
# The packages needed for this project of prediting flight delays
#from pyspark.sql import SQLContext
import pandas as pd
import numpy as np
#import pyspark.sql as sparksql
import findspark
from matplotlib import pyplot as plt
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
import seaborn as sns
from pyspark.sql.functions import isnan
from pyspark.sql.functions import col, sum
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('basics').getOrCreate()

In [2]:
# This is read the data from csv and create a data frame
df = spark.read.csv('DelayedFlight.csv',header=True,inferSchema=True)

In [3]:
#This show the top 5 rows in the data set (just to see the type of data if its numerical or txt value)
df.show(5)

+---+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+------------+------------+--------+-------------+-----------------+
| ID|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+---+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+------------+------------+--------+-------------+-----------------+
|  0|2009|    1|         3|        4|   2003|      1955|   2211|      2225|           WN|      335| N712SW|  

In [4]:
#show the data type in the dataset
df.printSchema()
print(df.head())

root
 |-- ID: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: integer (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: integer (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: integer (nullable = true)
 |-- CRSElapsedTime: integer (nullable = true)
 |-- AirTime: integer (nullable = true)
 |-- ArrDelay: integer (nullable = true)
 |-- DepDelay: integer (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiIn: integer (nullable = true)
 |-- TaxiOut: integer (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- 

In [5]:
#This show the number of rows in the dataset (raw data)
df.count()

183527

In [6]:
#Describe Operation using here is use to calculate the summary of all numerical columns in the data frame,hence the ext code was choose only few to explore the data set 
df.describe().show()

+-------+------------------+------+------+------------------+------------------+------------------+------------------+------------------+------------------+-------------+------------------+-------+------------------+------------------+------------------+------------------+-----------------+------+------+-----------------+-----------------+------------------+---------+----------------+------------------+------------------+------------------+-------------------+------------------+
|summary|                ID|  Year| Month|        DayofMonth|         DayOfWeek|           DepTime|        CRSDepTime|           ArrTime|        CRSArrTime|UniqueCarrier|         FlightNum|TailNum| ActualElapsedTime|    CRSElapsedTime|           AirTime|          ArrDelay|         DepDelay|Origin|  Dest|         Distance|           TaxiIn|           TaxiOut|Cancelled|CancellationCode|      CarrierDelay|      WeatherDelay|          NASDelay|      SecurityDelay| LateAircraftDelay|
+-------+------------------+----

In [7]:
# To show the summary statistics of target attribute is "ArrDelay" and other variables correlated attributes.
df.describe('ArrDelay', 'DepDelay', 'CarrierDelay', 'LateAircraftDelay', 'NASDelay', 'WeatherDelay').show()

+-------+------------------+-----------------+------------------+------------------+------------------+------------------+
|summary|          ArrDelay|         DepDelay|      CarrierDelay| LateAircraftDelay|          NASDelay|      WeatherDelay|
+-------+------------------+-----------------+------------------+------------------+------------------+------------------+
|  count|            182915|           183527|            120786|            120786|            120786|            120786|
|   mean|42.801492496514776|43.54567992720417|19.354469888894407|25.652600466941532|14.407298859139305|3.5300531518553475|
| stddev|55.934025300625585|52.14576384677182|42.468740186721114| 41.65849285132601| 33.92247559511985| 21.53771286273278|
|    min|               -67|                6|                 0|                 0|                 0|                 0|
|    max|              1525|             1355|              1158|               897|              1357|              1049|
+-------+-------

In [8]:
#Filter the target variables rows that contains values (NotNull)
df.filter(df.ArrDelay.isNull()).count() 
df.filter(df.ArrDelay.isNotNull()).count()

182915

In [9]:
#This shows the number of missing value in the target variable "ArrDelay" 
df.filter((df['ArrDelay'] == '') | df ['ArrDelay'].isNull() | isnan (df['ArrDelay'])) .count()

612

In [ ]:
#This code shows the summary of all the missing values in each field
from pyspark.sql.functions import lit
rows = df.count() 
summary = df.describe().filter(col("summary") == "count") 
summary.select(*((lit(rows)-col(c)).alias(c) for c in df.columns)).show()

In [ ]:
# This code is show the name of the fields or columns in the dataset (raw data)
len(df.columns), df.columns

In [ ]:
# This code is show the total number of columns in the dataset
len(df.columns)

In [ ]:
#Collection of the target variables
df.corr('ArrDelay','DepDelay')

In [10]:
# Selecting delay_col and assign to the target variables
delay_col= df.select('ArrDelay')
delay_col.show(5)

+--------+
|ArrDelay|
+--------+
|     -14|
|       2|
|      14|
|      34|
|      11|
+--------+
only showing top 5 rows



In [11]:
#Adding another columns in the data set and show only 5 rows
df.withColumn('Delay_times_5',df['ArrDelay']*5).show(5)


+---+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+------------+------------+--------+-------------+-----------------+-------------+
| ID|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|Delay_times_5|
+---+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+------------+------------+--------+-------------+-----------------+-------------+
|  0|2009|    1|         3|        4|   2003|      1955|   2211|   

In [ ]:
# Run the dataframe and new output missing the new column
df.show(5)

In [12]:
#Using filter within the dataframe
df.filter("ArrDelay < 15").show()

+---+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+------------+------------+--------+-------------+-----------------+
| ID|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+---+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+------------+------------+--------+-------------+-----------------+
|  0|2009|    1|         3|        4|   2003|      1955|   2211|      2225|           WN|      335| N712SW|  

In [13]:
#Arrival delay more than 15 minutes and selction from the carrier and flight number 
#also how long the departure delay since the arrival flight delay
df.filter("ArrDelay > 15").select('DepDelay','FlightNum', 'UniqueCarrier').show()

+--------+---------+-------------+
|DepDelay|FlightNum|UniqueCarrier|
+--------+---------+-------------+
|      34|     3920|           WN|
|      67|      509|           WN|
|      94|     1333|           WN|
|      28|        4|           WN|
|      51|       54|           WN|
|      32|      623|           WN|
|      87|      188|           WN|
|      82|      362|           WN|
|      82|      422|           WN|
|      56|     1056|           WN|
|      15|     3244|           WN|
|     315|      186|           WN|
|      45|     3154|           WN|
|      53|     1035|           WN|
|      10|      205|           WN|
|      23|     3430|           WN|
|      26|     1580|           WN|
|      30|     2195|           WN|
|      38|       54|           WN|
|      44|       23|           WN|
+--------+---------+-------------+
only showing top 20 rows



In [14]:
#Sample of the advance filter - values between 5 and 10 minutes delay
df.filter('ArrDelay <= 5' and 'ArrDelay >=10')\
  .select('FlightNum','DepDelay').distinct().count()

114244

In [ ]:
df.groupBy('UniqueCarrier').mean().show(5)

In [16]:
# To simplify things, let's split this into two steps. First, let's create a variable then order by age.
# Careful when using show()! Otherwise the variable type will change and you won't be able to order it. 
group_UniqueCarrier_df = df.groupBy('UniqueCarrier').mean()

# Note that we have to use 'avg(age)' instead of age. Why? Because when you use mean(), it changes the feature's name (as you can see below).
print("Sorted by ArrDelay")
group_UniqueCarrier_df.orderBy('avg(ArrDelay)').show()

# Let's see what this looks like in one line.
print("Sorted by DepDelay")
df.groupBy('UniqueCarrier').mean().orderBy('avg(DepDelay)').show()


Sorted by ArrDelay
+-------------+------------------+---------+----------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+----------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------+------------------+-------------------+------------------+--------------------+----------------------+
|UniqueCarrier|           avg(ID)|avg(Year)|avg(Month)|   avg(DayofMonth)|    avg(DayOfWeek)|      avg(DepTime)|   avg(CRSDepTime)|      avg(ArrTime)|   avg(CRSArrTime)|    avg(FlightNum)|avg(ActualElapsedTime)|avg(CRSElapsedTime)|      avg(AirTime)|     avg(ArrDelay)|     avg(DepDelay)|     avg(Distance)|       avg(TaxiIn)|      avg(TaxiOut)|avg(Cancelled)| avg(CarrierDelay)|  avg(WeatherDelay)|     avg(NASDelay)|  avg(SecurityDelay)|avg(LateAircraftDelay)|
+-------------+------------------+---------+----------+

+-------------+------------------+---------+----------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+----------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------+------------------+-------------------+------------------+--------------------+----------------------+
|UniqueCarrier|           avg(ID)|avg(Year)|avg(Month)|   avg(DayofMonth)|    avg(DayOfWeek)|      avg(DepTime)|   avg(CRSDepTime)|      avg(ArrTime)|   avg(CRSArrTime)|    avg(FlightNum)|avg(ActualElapsedTime)|avg(CRSElapsedTime)|      avg(AirTime)|     avg(ArrDelay)|     avg(DepDelay)|     avg(Distance)|       avg(TaxiIn)|      avg(TaxiOut)|avg(Cancelled)| avg(CarrierDelay)|  avg(WeatherDelay)|     avg(NASDelay)|  avg(SecurityDelay)|avg(LateAircraftDelay)|
+-------------+------------------+---------+----------+------------------+

In [18]:
group_UniqueCarrier_df = df.groupBy('ArrDelay').mean()

In [ ]:
print("Sorted by UniqueCarrier")
group_UniqueCarrier_df.orderBy('avg(ArrDelay)').show(2)

Sorted by UniqueCarrier


In [15]:
print("Sorted by UniqueCarrier")
df.groupBy('UniqueCarrier').mean().orderBy('avg(DepDelay)').show(2)

Sorted by UniqueCarrier
+-------------+------------------+---------+----------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+----------------------+-------------------+------------------+------------------+------------------+-----------------+-----------------+------------------+--------------+------------------+------------------+------------------+------------------+----------------------+
|UniqueCarrier|           avg(ID)|avg(Year)|avg(Month)|   avg(DayofMonth)|    avg(DayOfWeek)|      avg(DepTime)|   avg(CRSDepTime)|      avg(ArrTime)|   avg(CRSArrTime)|   avg(FlightNum)|avg(ActualElapsedTime)|avg(CRSElapsedTime)|      avg(AirTime)|     avg(ArrDelay)|     avg(DepDelay)|    avg(Distance)|      avg(TaxiIn)|      avg(TaxiOut)|avg(Cancelled)| avg(CarrierDelay)| avg(WeatherDelay)|     avg(NASDelay)|avg(SecurityDelay)|avg(LateAircraftDelay)|
+-------------+------------------+---------+----------+-------